In [15]:
import sys
sys.path.append('../../code/')
from utils.preprocess import read_json
from Ques_pair import Ques_pair, Type

In [4]:
ref_file = read_json("../../datasets/qald9plus/qald_9_plus_test_wikidata.json")
pred_file = read_json("/Users/mengshima/Downloads/en.json") # mt5 linguistic 11 languages

In [5]:
ref_ques_list = [ref for ref in ref_file["questions"]]
pred_ques_list = [pred for pred in pred_file["questions"]]

In [6]:
n = 5
for ref, pred in zip(ref_ques_list[:n], pred_ques_list[:n]):
    ques_pair = Ques_pair(ref, pred)
    ques_pair.print(print_answer=True)
    print()

id: 99
question: What is the time zone of Salt Lake City?
ref sparql: SELECT DISTINCT ?o1 WHERE { wd:Q23337 wdt:P421 ?o1 . }
pred sparql: SELECT DISTINCT ?uri WHERE { wd:Q187893 wdt:P421 ?uri }
ref_answer:
 [{'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3134980'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2212'}}]
pred_answer:
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q5762'}}]


id: 98
question: Who killed Caesar?
ref sparql: SELECT DISTINCT ?o1 WHERE { wd:Q1048 wdt:P157 ?o1 . }
pred sparql: SELECT DISTINCT ?uri WHERE { wd:Q692 wdt:P570 ?uri . }
ref_answer:
 [{'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q172248'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q207370'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q294846'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1228715'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/ent

In [7]:
entity_f1_sum = 0
relation_f1_sum = 0
num_of_ques = len(ref_ques_list)

for ref, pred in zip(ref_ques_list, pred_ques_list):
    ques_pair = Ques_pair(ref, pred)
    entity_f1_sum += ques_pair.calculate_entity_f1()
    relation_f1_sum += ques_pair.calculate_relation_f1()

macro_entity_f1 = entity_f1_sum / num_of_ques
macro_relation_f1 = relation_f1_sum / num_of_ques

In [8]:
macro_entity_f1

0.1744281045751634

In [9]:
macro_relation_f1

0.2720588235294118

In [10]:
# check reason of no answer
counter = 0

for ref, pred in zip(ref_ques_list, pred_ques_list):
    ques_pair = Ques_pair(ref, pred)
    if ques_pair.pred_answer==[]:
        ques_pair.print()
        counter += 1

print(str(counter) + " SPARQL queries don't give an answer")

# counter = sum([ques_pair.print() or 1 for ref, pred in zip(ref_ques_list, pred_ques_list) if (ques_pair := Ques_pair(ref, pred)).pred_answer == []])
# print(counter)


id: 84
question: Which American presidents were in office during the Vietnam War?
ref sparql: SELECT DISTINCT ?o WHERE { wd:Q8740 wdt:P580 ?startWar . wd:Q8740 wdt:P582 ?endWar . wd:Q30 <p:P6> ?blank . ?blank ps:P6 ?o . ?blank pq:P580 ?start . ?blank pq:P582 ?end . FILTER (?startWar < ?start && ?endWar > ?end)} 
pred sparql: SELECT DISTINCT ?uri WHERE { ?uri wdt:P39 wd:Q9696 ; wdt:P20 wd:Q20 . }

id: 81
question: Butch Otter is the governor of which U.S. state?
ref sparql: SELECT ?res WHERE { ?res wdt:P6 wd:Q39593 . }
pred sparql: SELECT DISTINCT ?uri WHERE { wd:Q4970706 wdt:P6 ?uri . } 

id: 56
question: How many seats does the home stadium of FC Porto have?
ref sparql: SELECT DISTINCT ?num WHERE { wd:Q128446 wdt:P115/wdt:P1083 ?num }
pred sparql: SELECT DISTINCT ?uri WHERE { wd:Q483110 wdt:P131 ?uri . }

id: 49
question: Which frequent flyer program has the most airlines?
ref sparql: SELECT ?frequentflyer WHERE { ?airlines wdt:P31 wd:Q46970 . ?airlines p:P4446/ps:P4446 ?frequentflyer

In [11]:
# check correct answers
counter = 0

for ref, pred in zip(ref_ques_list, pred_ques_list):
    ques_pair = Ques_pair(ref, pred)
    if ques_pair.pred_answer==ques_pair.ref_answer:
        ques_pair.print()
        counter += 1

print(str(counter) + " questions are 100% correct")

id: 81
question: Butch Otter is the governor of which U.S. state?
ref sparql: SELECT ?res WHERE { ?res wdt:P6 wd:Q39593 . }
pred sparql: SELECT DISTINCT ?uri WHERE { wd:Q4970706 wdt:P6 ?uri . } 

id: 149
question: Which U.S. state has the highest population density?
ref sparql: SELECT ?uri { ?uri wdt:P31 wd:Q35657 . ?uri wdt:P1082 ?population . ?uri wdt:P2046 ?area . BIND((?population/?area) AS ?density ) } ORDER BY DESC(?density) LIMIT 1
pred sparql: SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q35657 ; wdt:P2046 ?area ; wdt:P1082 ?population . BIND((?population/?area) AS ?density) . } ORDER BY DESC(?density) LIMIT 1

id: 21
question: Who created English Wikipedia?
ref sparql: SELECT ?uri WHERE { wd:Q52 wdt:P170 ?uri . ?uri wdt:P31 wd:Q5 . }
pred sparql: SELECT DISTINCT ?uri WHERE { wd:Q52 wdt:P112 ?uri . }

id: 141
question: Which languages are spoken in Estonia?
ref sparql: SELECT * WHERE { wd:Q191 wdt:P2936 ?lang . }
pred sparql: SELECT * WHERE { wd:Q191 wdt:P2936 ?lang . }

id: 45

In [23]:
# check ASK queries
count_all = 0
count_ask = 0

for ref, pred in zip(ref_ques_list, pred_ques_list):
    ques_pair = Ques_pair(ref, pred)
    if ques_pair.type == Type.BOOLEAN:
        ques_pair.print(print_answer=True)
        count_all += 1
        if "ASK" in ques_pair.pred_sparql:
            count_ask += 1

print(f'''{count_ask} of {count_all} predicted queries contain "ASK"''')

id: 117
question: Is Pamela Anderson a vegan?
ref sparql: ASK where { wd:Q83325 wdt:P1576 wd:Q83364}
pred sparql: ASK WHERE { wd:Q134041 wdt:P106 wd:Q40050 . }
ref_answer:
 True
pred_answer:
 False

id: 79
question: Are there any castles in the United States?
ref sparql: ASK WHERE { { ?uri wdt:P31 wd:Q23413. } UNION { ?uri wdt:P31 ?castlesubtype . ?castlesubtype wdt:P279* wd:Q23413 } ?uri wdt:P17 wd:Q30 }
pred sparql: SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q1207989 ; wdt:P17 wd:Q30 . }
ref_answer:
 False
pred_answer:
 []

id: 92
question: Is the wife of President Obama called Michelle?
ref sparql: ASK WHERE { wd:Q76 wdt:P26 ?obj . ?obj rdfs:label ?label FILTER( REGEX( ?label, "Michelle" ) ) }
pred sparql: ASK WHERE { wd:Q76 wdt:P26 ?wife . ?wife rdfs:label ?name . FILTER regex(?name, "Michelle") }
ref_answer:
 True
pred_answer:
 True

2 of 3 predicted queries contain "ASK"


In [18]:
# check COUNT queries
for ref, pred in zip(ref_ques_list, pred_ques_list):
    ques_pair = Ques_pair(ref, pred)
    if ques_pair.type == Type.COUNT:
        ques_pair.print(print_answer=True)

id: 42
question: Which countries have places with more than two caves?
ref sparql: SELECT DISTINCT ?uri WHERE { ?cave wdt:P31 wd:Q35509 ; wdt:P17 ?uri . } GROUP BY ?uri HAVING (COUNT(?cave) > 2)
pred sparql: SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q6256 . ?uri wdt:P37 ?lake . ?uri wdt:P31 wd:Q5287 . } GROUP BY ?uri HAVING(COUNT(?lake)>
ref_answer:
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q233'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q236'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q242'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q241'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q252'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q34020'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q46395'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q219060'}}, {'uri': {'type': 'uri', 'value': 'http://www

In [22]:
# check RANK queries
count_all = 0
count_contains_order = 0

for ref, pred in zip(ref_ques_list, pred_ques_list):
    ques_pair = Ques_pair(ref, pred)
    if ques_pair.type == Type.RANK:
        ques_pair.print(print_answer=True)
        count_all += 1
        if "ORDER" in ques_pair.pred_sparql:
            count_contains_order += 1

print(f'''{count_contains_order} of {count_all} predicted queries contain "ORDER"''')

id: 86
question: What is the highest mountain in Germany?
ref sparql: SELECT ?uri WHERE { ?uri wdt:P31 wd:Q8502 ; wdt:P2044 ?elevation ; wdt:P17 wd:Q183 . } ORDER BY DESC(?elevation) LIMIT 1
pred sparql: SELECT ?res WHERE { ?res wdt:P31 wd:Q8502 . ?res wdt:P131+ wd:Q1202 . ?res wdt:P2044 ?height . } ORDER BY DESC(?height) LIMIT 1
ref_answer:
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3375'}}]
pred_answer:
 [{'res': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q164561'}}]

id: 49
question: Which frequent flyer program has the most airlines?
ref sparql: SELECT ?frequentflyer WHERE { ?airlines wdt:P31 wd:Q46970 . ?airlines p:P4446/ps:P4446 ?frequentflyer . } GROUP BY ?frequentflyer ORDER BY DESC(COUNT(?airlines)) LIMIT 1
pred sparql: SELECT DISTINCT ?uri WHERE { ?uri wdt:P31/wdt:P279* wd:Q46970 } ?uri wdt:P106 ?uri . } GROUP BY ?uri ORDER BY DESC(COUNT(DISTINCT ?uri) LIMIT 1
ref_answer:
 [{'frequentflyer': {'type': 'uri', 'value': 'http://www.wikidata.

# question, sparql query and answers

## id: 99

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

Wrong entity: Q187893(San Pedro de Atacama)

Q23337(Salt Lake City) is not in trianing data

## id: 98

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌
predicted entity: Q1203(John Lennon)

Q1048(Julius Caesar) is not in training data

## id: 86

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ❌

A tuple is wrong, it gives the highest mountain in Q1202(Saxony), the rest parts are correct

It learns from 201 in training set: "What is the highest mountain in Saxony Germany?"

## id: 84

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


absolutly no sense

out-of-scope: Q8740

training set 324: "Was U.S. president Jackson involved in a war?"

## id: 81

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

it get the answer of "who is the governor of Q1214(Wyoming)"

Q39593 is not in the training data

training set 66: "Who is the governor of Wyoming?"

## id:66

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ❌
- entity ✅
- relation ✅
- answer ❌

get the birth date of Rachel Stevens correctly, but miss the part of find other artists

out-of-scope: Q483501(artists)

## id:64

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ❌

out-of-scope: P106(occupation)

However, P194(doctoral advisor) is a relation which is plausible

training set 337: "Who was the doctoral supervisor of Albert Einstein?"

## id: 56

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

floors above ground (P1101)

out-of-scope: FC Porto (Q128446)

FC Porto (Q128446) and maximum capacity (P1083) are not in training data

## id: 49

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ❌

understand that answer should is related to airlines, but fail to understand "most" and "frequent flyer program"

out-of-scope: reward program (P4446)

training 58: "How many airlines are there?"|

## id: 44

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 163: "Which Greek parties are pro-European?"

out-of-scope: constitutional monarchy (Q41614)

## id: 42

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 57: "Which rivers flow into a German lake?"

out-of-scope: cave (Q35509)

## id: 4

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: airport (Q1248784)

## id: 32

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ✅

training 151: "What is the birth name of Angela Merkel?"

## id: 31

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

training 300: "Who is the mayor of Rotterdam?"

## id: 29

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 102: "Which countries adopted the Euro?"

out-of-scope: contains the administrative territorial entity (P150)

## id: 68

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

out-of-scope: Bill Gates (Q5284)

## id: 22

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 144: "How many children does Eddie Murphy have?"

out-of-scope: Jacques Cousteau (Q83233)

doesn't understand P40/P40 for grand-children

## id: 214

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: sport (P641) skateboarding (Q842284)

## id: 210

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 163: "Which monarchs were married to a German?"

out-of-scope: Great Britain (Q161885)

## id: 199

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

training 51: Give me all Danish movies.

out-of-scope: Argentina (Q414)

## id: 203

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: Michael Jackson (Q2831)

## id: 197

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ❌

understand "latest" with ORDER

## id: 196

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

training 255: "Which species does an elephant belong?"

out-of-scope: millipede (Q11185669)

## id: 188

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ❌

training 220: "Who is the editor of Forbes?"

## id: 187

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 275: "List the children of Margaret Thatcher."

out-of-scope: Amanda Palmer (Q240377)

## id: 176

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 335: "Who wrote the book The Pillars of the Earth?"

out-of-scope: The Storm on the Sea of Galilee (Q2246489)

misinterpreted creator (P170) as author (P50)

## id: 62

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

training 65: "What is the highest place of the Urals?"

out-of-scope: Karakoram (Q5469)

## id: 173

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

training 138: "When did Latvia join the EU?"

out-of-scope: Finland (Q33)

## id: 168

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

training 412: "Give me all actors starring in Last Action Hero."

out-of-scope: The Big Bang Theory (Q8539)

## id: 166

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 328: "Which scientist is known for the Manhattan Project and the Nobel Peace Prize?"

out-of-scope: computer scientist (Q82594) Academy Awards (Q19020)

## id: 160

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

training 163: "Which Greek parties are pro-European?"

out-of-scope: Harry Potter (Q8337)

## id: 158

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))


- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ✅

training 33: "Give me all writers that won the Nobel Prize in literature."

## id: 157

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 389: "Which movies starring Mickey Rourke were directed by Guy Ritchie?"

out-of-scope: Lovesick (Q12125895)

## id: 150

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 398: "Who is the president of Eritrea?"

out-of-scope: Montenegro (Q236)

## id: 149

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ✅

training 16: "Which state of the USA has the highest population density?"

## id: 25

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ❌
- entity ❌
- relation ❌
- answer ❌

training 149: "What is the longest river?"

## id: 143

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: local dialing code (P473)

## id: 140

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

training 140: "How many politicians graduated from Columbia University?"

out-of-scope: Ivy League (Q49113)

## id: 139

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ❌
- entity ❌
- relation ✅
- answer ❌

training 82: "Which actors were born in Germany?"

out-of-scope: surfer (Q13561328)

## id: 137

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 42: "Give me all Swiss non-profit organizations."

## id: 136

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 156: "How heavy is Jupiter's lightest moon?"

out-of-scope: Mars (Q111) wdt:P398

## id: 134

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ✅

out-of-scope: said to be the same as (P460)

## id: 132

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: Elon Musk (Q317521)

## id: 128

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 200: "Who created Wikipedia?"

out-of-scope: WikiLeaks (Q359)

## id: 124

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 7: "Where did Abraham Lincoln die?"

out-of-scope: Carlo Giuliani (Q164791)

## id: 122

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ✅

training 254: "Who was the wife of U.S. president Lincoln?"

## id: 111

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

training 163: "Which Greek parties are pro-European?"

out-of-scope: 
- Bertrand Russell (Q33760)

doesn't understand "How many" as "COUNT"

## id: 117

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 107: "Was Margaret Thatcher a chemist?"

out-of-scope: 
- Pamela Anderson (Q83325)
- lifestyle (P1576)
- vegetarianism (Q83364)

## id: 59

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

training 23: "Which spaceflights were launched from Baikonur?"

out-of-scope: 
- type of orbit (P522)
- heliocentric orbit (Q737809)

## id: 113

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

## id: 104

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 27: 
"Who is the daughter of Ingrid Bergman married to?"

out-of-scope: 
- Robert F. Kennedy (Q25310)

## id: 10

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 140: 
"How many politicians graduated from Columbia University?"

out-of-scope: 
- University of Amsterdam (Q214341)
- students count (P2196)

## id: 1

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ❌

out-of-scope: 
- total revenue (P2139)

## id: 178

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 50: 
"How many films did Hal Roach produce?"

out-of-scope: 
- based on (P144)
- characters (P674)

## id: 129

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: 
- Bart Simpson (Q5480)

## id: 183

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 152: 
"Who has Tom Cruise been married to?"

## id: 181

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 81: 
"To which countries does the Himalayan mountain system extend?"

out-of-scope: 
- Yenisey (Q78707)

## id: 7

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 14: 
"Which other weapons did the designer of the Uzi develop?"

out-of-scope: 
- automobile model series (Q59773381)
- automobile manufacturer (Q786820)


## id: 135

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 252: 
"When was John Adams born?"

out-of-scope: 
- Michael Jackson (Q2831)

## id: 50

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌✅
- answer ❌

predicted query from training 400: 
"What is the highest mountain in Australia?"

out-of-scope: 
- volcano (Q8072)

## id: 39

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ❌
- entity ✅
- relation ✅
- answer ✅

predicted query from training 121: 
"Which poet wrote the most books?"

## id: 71

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 23: 
"Which spaceflights were launched from Baikonur?"

out-of-scope: 
- Mars rover (Q389459)

## id: 213

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 51: 
"Give me all Danish movies."

out-of-scope: 
- Czechoslovakia (Q33946)

## id: 209

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 41: 
"Give me all cosmonauts."

## id: 105

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 57: 
"Which rivers flow into a German lake?"

out-of-scope: 
- volcano (Q8072)

## id: 52

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ✅

predicted query from training 171: 
"Give me all movies with Tom Cruise."

## id: 102

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌✅
- answer ❌

predicted query from training 210: 
"Who created Batman?"

out-of-scope: 
- Count Dracula (Q3266236)

## id: 21

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ✅

predicted query from training 200: 
"Who created Wikipedia?"

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

## id: 145

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 365: 
"Who created Family Guy?"

out-of-scope: 
- ALDI (Q125054)
- has subsidiary (P355)

## id: 154

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ❌

predicted query from training 109: 
"Give me all books written by Danielle Steel."

## id: 198

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 339: 
"Who was Vincent van Gogh inspired by?"

## id: 162

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 304: 
"How high is the Yokohama Marine Tower?"

out-of-scope: 
- Chiemsee (Q4138)

## id: 212

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅❌
- answer ❌

predicted query from training 253: 
"Which companies produce hovercrafts?"

out-of-scope: 
- aerospace industry (Q3477363)
- health care industry (Q15067276)

## id: 96

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ❌
- entity ❌
- relation ❌
- answer ❌

predicted query from training 82: 
"Which actors were born in Germany?"

out-of-scope: 
- surfer (Q13561328)

## id: 40

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: 
- Victoria (Q9439)

## id: 79

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: 
- castle (Q23413)

## id: 141

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ✅

predicted query from training 187: 
"Which languages are spoken in Estonia?"

## id: 80

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 185: 
"Give me all breeds of the German Shepherd dog."

out-of-scope: 
- birds (Q5113)
- IUCN conservation status (P141)
- Critically Endangered (Q219127)

## id: 159

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 262: 
"With how many countries Iran has borders?"

out-of-scope: 
- Rhine (Q584)

## id: 107

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 72: 
"Which pope succeeded John Paul II?"

out-of-scope: 
- Elizabeth II (Q9682)

## id: 163

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 103: 
"Give me all types of eating disorders."

out-of-scope: 
- chemical element (Q11344)

## id: 211

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ❌
- entity ❌
- relation ❌
- answer ❌

predicted query from training 68: 
"Give me all world heritage sites designated within the **past two years**."

## id: 51

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 356: 
"When did Operation Overlord commence?"

out-of-scope: 
- Worst Case Scenario (Q6667101)

## id: 144

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 152: 
"Who has Tom Cruise been married to?"

out-of-scope: 
- Lance Bass (Q443120)

## id: 131

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: 
- Pakistan (Q843)

## id: 120

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌✅
- answer ❌

predicted query from training 152: 
"Who has Tom Cruise been **married to**?"

out-of-scope: 
- Bill Clinton (Q1124)

## id: 164

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 20: 
"Give me the Apollo 14 astronauts."

out-of-scope: 
- Apollo 11 (Q43653)

## id: 151

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ❌
- entity ✅
- relation ✅
- answer ❌

predicted query from training 175: 
"Give me all B-sides of the Ramones"

## id: 63

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 365: 
"Who created Family Guy?"

out-of-scope: 
- nickname (P1449)

## id: 27

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 57: 
"**Which rivers flow into** a German lake?"

out-of-scope: 
- North Sea (Q1693)

## id: 156

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ❌✅

predicted query from training 172: 
"In which U.S. state is Fort Knox located?"

## id: 207

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅❌
- answer ❌

predicted query from training 169: 
"Which Chess players **died in the same place they were born in?**"

out-of-scope: 
- British Isles (Q38272)

## id: 24

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 24: 
"Give me all **federal chancellors** of Germany."

out-of-scope: 
- Emperor of China (Q268218)

## id: 133

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 210: 
"Who created Batman?"

out-of-scope: 
- Teenage Mutant Ninja Turtles (Q1381762)

## id: 103

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

Where: located in the administrative territorial entity (P131)

out-of-scope: 
- Piccadilly (Q1124023)

## id: 165

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌✅
- answer ❌

predicted query from training 266: 
"Did Arnold Schwarzenegger attend a university?",
but change "ASK" to "SELECT"

## id: 108

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 368: 
"When did the Ming dynasty **dissolve**?"

out-of-scope: 
- Paraguay (Q733)

## id: 15

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ❌
- entity ❌
- relation ❌
- answer ❌

out-of-scope: 
- National Basketball Association (Q155223)

## id: 45

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ✅

predicted query from training 7: 
"Where did Abraham Lincoln die?"

## id: 82

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 92: 
"When were the Hells Angels **founded**?"

out-of-scope: 
- Jack Wolfskin (Q536133)

## id: 9

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

## id: 201

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: 
- Plzeňský Prazdroj (Q948831)

## id: 171

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 356: 
"When did Operation Overlord commence?"

out-of-scope: 
- Boston Tea Party (Q19024)

## id: 182

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 185: 
"Give me all breeds of the German Shepherd dog."

out-of-scope: 
- IUCN conservation status (P141)
- Critically Endangered (Q219127)

## id: 13

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ❌✅

predicted query from training 17: 
"Which monarchs were married to a German?"

## id: 114

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 156: 
"How heavy is Jupiter's lightest moon?"

out-of-scope: 
- diameter (P2386)

## id: 92

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ✅

predicted query from training 227: 
"Is the wife of president Obama called Michelle?"

## id: 194

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌✅
- relation ❌✅
- answer ❌

predicted query from training 172: 
"In which **U.S. state** is Fort Knox located?"

out-of-scope: 
- short name (P1813)

## id: 175

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 239: 
"When was the death of Shakespeare?"

out-of-scope: 
- Muhammad (Q9458)

## id: 60

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 66: 
"**Who is the governor of** Wyoming?"

but relations are not the same

out-of-scope: 
- Texas (Q1439)

## id: 110

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 282: 
"Which films did Stanley Kubrick direct?"

out-of-scope: 
- Akira Kurosawa (Q8006)

## id: 206

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: 
- San Antonio (Q975)
- battle (Q178561)

## id: 97

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ❌

predicted query from training 141: 
"Give me the birthdays of all actors of the television show Charmed."

## id: 169

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ✅
- relation ✅❌
- answer ❌

predicted query from training 113: 
"Give me all libraries established earlier than 1400."

## id: 38

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌✅
- answer ❌

predicted query from training 143: 
"Give me a list of all lakes in Denmark."

out-of-scope: 
- Frisian Islands (Q76936)

## id: 192

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: 
- The Scream (Q471379)
- art museum (Q207694)

## id: 78

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 92: 
"When were the Hells Angels founded?"

out-of-scope: 
- De Beers (Q471891)

## id: 19

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ✅
- relation ❌
- answer ❌

predicted query from training 77: 
"Where was JFK assassinated?"

## id: 190

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅❌
- answer ❌
  
out-of-scope: 
- Juan Carlos I of Spain (Q19943)


## id: 174

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 238: 
"Who is the author of the interpretation of dreams?"

out-of-scope: 
- A Song of Ice and Fire (Q45875)

## id: 189

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌❌✅❌
- answer ❌

predicted query from training 130: 
"For which label did Elvis record his **first album**?"

out-of-scope: 
- The Beatles (Q1299)
- recorded at studio or venue (P483)

## id: 152

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ✅❌
- relation ✅❌
- answer ❌

predicted query from training 181: 
"Show me all the breweries in Australia."

## id: 94

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ✅
- answer ❌

predicted query from training 239: 
"When was the death of Shakespeare?"

out-of-scope: 
- Diana, Princess of Wales (Q9685)

## id: 8

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 37: 
"Who developed Minecraft?"

out-of-scope: 
- Intel (Q248)

## id: 95

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ❌
- entity ❌
- relation ❌
- answer ❌

## id: 119

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ✅
- relation ✅
- answer ✅

predicted query from training 126: 
"Which instruments does Cat Stevens play?"

## id: 115

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 57: 
"Which rivers flow into a German lake?" (modified)

out-of-scope: 
- South Carolina (Q1456)

understand "how many" as "COUNT"✅

## id: 116

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

out-of-scope: 
- skos:altLabel

## id: 138

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ✅❌
- relation ✅❌❌
- answer ❌

out-of-scope: 
- Africa (Q15)

## id: 177

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 48: 
"Which bridges cross the Seine?"

out-of-scope: 
- Manhattan Bridge (Q125050)

## id: 101

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 85: 
"How many employees does Google have?"

out-of-scope: 
- Mark Zuckerberg (Q36215)
- company (Q783794)

## id: 87

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ❌
- entity ✅
- relation ❌
- answer ❌

predicted query from training 163: 
"Which poet wrote the **most** books?"

out-of-scope: 
- number of pages (P1104)

## id: 148

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 189: 
"What is the largest city in america?"

## id: 43

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ✅
- relation ✅❌
- answer ❌

predicted query from training 62: 
"Which companies have more than 1 million employees?"

## id: 179

In [ ]:
next(gold_ques_gen).print_along_with(next(pred_ques_gen))

- SPARQL syntax ✅
- entity ❌
- relation ❌
- answer ❌

predicted query from training 23: 
"Which spaceflights were launched from Baikonur?"

out-of-scope: 
- ship (Q11446)
- participant (P710)
- Christopher Columbus (Q7322)

# Summary

## Out-of-Scope questions

99, 98, 84, 81, 66, 
64, 56, 49, 44, 42, 
4, 29, 68, 22, 214, 
210, 199, 203, 196, 187,
176, 62, 173, 168, 166,
160, 157, 150, 143, 140,
139, 136, 132, 128, 124,
111, 117, 59, 104, 10,
1, 178, 129, 181, 7,
135, 50, 71, 213, 105,
102, 145, 162, 212, 96,
40, 80, 159, 107, 163,
51, 144, 131, 120, 164,
63, 27, 207, 24, 133,
103, 108, 15, 82, 201,
171, 182, 114, 194, 175,
60, 110, 206, 38, 192,
78, 190, 174, 189, 115,
116, 138, 177, 101, 87, 179

## In-Scope questions

86, 32, 31, 197, 188,
149, 25, 137, 122, 113,
183, 209, 21, 154, 198,
211, 156, 165, 9, 13, 
97, 19, 152, 94, 8,
95, 148, 43


## In training set

158, 39, 52, 141, 151,
45, 92, 169, 119

## issues

- Some SPARQL learned from training set
- most of predicted SPARQL are more or less related to the question, even the predicted answer is direct from training set
- loss the ability of generalization 
- Some questions in test set also appear in training set, but with different SPARQL
- fine-tuned model can write SPARQL, but most of them does not give any answer
- since the training data are non-preprocessed SPARQL, some prefixes are too long
- hard to generate SPARQL with "http://www.wikidata.org..."

## improvements

- preprocess SPARQL
- less training steps
- pretrain on lc-quad to gain more entities and relations